# 🚀 算法深度探索实验

## 📋 实验目标
这个notebook不是让您运行现成代码，而是通过一系列**探索性任务**帮助您真正理解算法的本质。

### 🎯 您将学到：
1. **参数的作用机制** - 为什么要这样调参？
2. **算法的适用边界** - 什么时候算法会失效？
3. **真实问题解决** - 如何应对实际挑战？
4. **创新思维培养** - 如何改进和优化算法？

### ⚠️ 学习原则：
- **不要只运行代码** - 先思考，再验证
- **记录每个发现** - 建立自己的知识体系
- **挑战现有结论** - 提出自己的假设并验证
- **联系实际应用** - 思考这些发现在实际中的意义

## 🔧 实验准备

让我们先设置好实验环境，然后开始真正的探索之旅！

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.inspection import DecisionBoundaryDisplay
import warnings
warnings.filterwarnings('ignore')

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# 加载数据
def load_data():
    """加载乳腺癌和鸢尾花数据集"""
    # 乳腺癌数据集
    X_bc, y_bc = load_svmlight_file("../data/breast-cancer_scale")
    X_bc = X_bc.toarray()
    y_bc = np.where(y_bc == 2, 0, 1)  # 转换为0,1
    
    # 鸢尾花数据集
    X_iris, y_iris = load_svmlight_file("../data/iris.scale")
    X_iris = X_iris.toarray()
    y_iris = y_iris - 1  # 转换为0,1,2
    
    return (X_bc, y_bc), (X_iris, y_iris)

# 数据预处理
(X_bc, y_bc), (X_iris, y_iris) = load_data()

# 分割数据
X_bc_train, X_bc_test, y_bc_train, y_bc_test = train_test_split(
    X_bc, y_bc, test_size=0.3, random_state=42, stratify=y_bc
)
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42, stratify=y_iris
)

# 标准化
scaler_bc = StandardScaler()
X_bc_train = scaler_bc.fit_transform(X_bc_train)
X_bc_test = scaler_bc.transform(X_bc_test)

scaler_iris = StandardScaler()
X_iris_train = scaler_iris.fit_transform(X_iris_train)
X_iris_test = scaler_iris.transform(X_iris_test)

print("✅ 实验环境准备完成！")
print(f"乳腺癌数据集：训练{X_bc_train.shape}，测试{X_bc_test.shape}")
print(f"鸢尾花数据集：训练{X_iris_train.shape}，测试{X_iris_test.shape}")

## 🎯 探索任务1：参数的深度理解

### 🤔 思考问题：
1. **C参数的本质是什么？** 它控制模型的什么特性？
2. **gamma参数如何影响决策边界？** 为什么它对RBF核如此重要？
3. **参数之间是否存在交互效应？** 某些参数组合是否会相互增强或抵消？

### 🔬 实验任务：
完成下面的代码，观察参数变化对模型性能的影响，**并记录您的发现**

In [ ]:
# 📊 任务1.1: C参数的深度探索
# 🎯 目标：理解C参数如何控制模型的复杂度和过拟合

def explore_c_parameter(X_train, X_test, y_train, y_test, dataset_name):
    """探索C参数对SVM性能的影响"""
    
    # TODO: 定义更细粒度的C值范围
    c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]  # 可以调整这个范围
    
    train_accs = []
    test_accs = []
    
    print(f"🔍 正在探索{dataset_name}上C参数的影响...")
    
    for c in c_values:
        # TODO: 训练RBF SVM，记录训练和测试准确率
        svm = SVC(C=c, kernel='rbf', gamma='scale', random_state=42)
        svm.fit(X_train, y_train)
        
        train_acc = accuracy_score(y_train, svm.predict(X_train))
        test_acc = accuracy_score(y_test, svm.predict(X_test))
        
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        
        print(f"C={c:.3f}: 训练准确率={train_acc:.4f}, 测试准确率={test_acc:.4f}")
    
    # 绘制结果
    plt.figure(figsize=(10, 6))
    plt.semilogx(c_values, train_accs, 'bo-', label='训练准确率', linewidth=2, markersize=8)
    plt.semilogx(c_values, test_accs, 'ro-', label='测试准确率', linewidth=2, markersize=8)
    plt.xlabel('C参数值 (对数尺度)')
    plt.ylabel('准确率')
    plt.title(f'{dataset_name}: C参数对模型性能的影响')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # TODO: 分析并记录您的发现
    print("\n💡 分析要点：")
    print("1. 什么时候出现欠拟合？（训练和测试准确率都低）")
    print("2. 什么时候出现最佳性能？（测试准确率最高）")
    print("3. 什么时候出现过拟合？（训练准确率高，测试准确率低）")
    print("4. 最优的C值范围是什么？")
    
    return c_values, train_accs, test_accs

# 在两个数据集上探索C参数
print("="*60)
print("🎯 探索任务1.1: C参数的深度理解")
print("="*60)

# 乳腺癌数据集
c_vals_bc, train_accs_bc, test_accs_bc = explore_c_parameter(
    X_bc_train, X_bc_test, y_bc_train, y_bc_test, "乳腺癌数据集"
)

# 鸢尾花数据集
c_vals_iris, train_accs_iris, test_accs_iris = explore_c_parameter(
    X_iris_train, X_iris_test, y_iris_train, y_iris_test, "鸢尾花数据集"
)

In [ ]:
# 📊 任务1.2: Gamma参数的非线性效应
# 🎯 目标：理解gamma如何控制RBF核的影响范围

def explore_gamma_parameter(X_train, X_test, y_train, y_test, dataset_name, best_c=1):
    """探索gamma参数对SVM性能的影响"""
    
    # TODO: 定义gamma值范围
    gamma_values = ['auto', 'scale', 0.001, 0.01, 0.1, 1, 10, 100]
    
    train_accs = []
    test_accs = []
    
    print(f"🔍 正在探索{dataset_name}上gamma参数的影响 (C={best_c})...")
    
    for gamma in gamma_values:
        try:
            # TODO: 训练SVM，处理不同gamma值
            svm = SVC(C=best_c, kernel='rbf', gamma=gamma, random_state=42)
            svm.fit(X_train, y_train)
            
            train_acc = accuracy_score(y_train, svm.predict(X_train))
            test_acc = accuracy_score(y_test, svm.predict(X_test))
            
            train_accs.append(train_acc)
            test_accs.append(test_acc)
            
            print(f"gamma={gamma}: 训练准确率={train_acc:.4f}, 测试准确率={test_acc:.4f}")
        except Exception as e:
            print(f"gamma={gamma}: 出现错误 - {e}")
            train_accs.append(0)
            test_accs.append(0)
    
    # 绘制结果（跳过'auto'和'scale'，因为它们是字符串）
    numeric_gamma = [g for g in gamma_values if isinstance(g, (int, float))]
    if numeric_gamma:
        numeric_train_accs = [train_accs[i] for i, g in enumerate(gamma_values) if g in numeric_gamma]
        numeric_test_accs = [test_accs[i] for i, g in enumerate(gamma_values) if g in numeric_gamma]
        
        plt.figure(figsize=(10, 6))
        plt.semilogx(numeric_gamma, numeric_train_accs, 'bo-', label='训练准确率', linewidth=2, markersize=8)
        plt.semilogx(numeric_gamma, numeric_test_accs, 'ro-', label='测试准确率', linewidth=2, markersize=8)
        plt.xlabel('Gamma参数值 (对数尺度)')
        plt.ylabel('准确率')
        plt.title(f'{dataset_name}: Gamma参数对模型性能的影响')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    
    print("\n💡 分析要点：")
    print("1. 太小的gamma值会导致什么？")
    print("2. 太大的gamma值会导致什么？")
    print("3. 'scale'和'auto'的表现如何？")
    print("4. 最优的gamma范围是什么？")
    
    return gamma_values, train_accs, test_accs

# TODO: 使用刚才找到的最佳C值
best_c_bc = 1  # 根据上一步的结果调整
best_c_iris = 1  # 根据上一步的结果调整

print("\n" + "="*60)
print("🎯 探索任务1.2: Gamma参数的非线性效应")
print("="*60)

# 在乳腺癌数据集上探索gamma
gamma_vals_bc, train_accs_bc_gamma, test_accs_bc_gamma = explore_gamma_parameter(
    X_bc_train, X_bc_test, y_bc_train, y_bc_test, "乳腺癌数据集", best_c_bc
)

# 在鸢尾花数据集上探索gamma
gamma_vals_iris, train_accs_iris_gamma, test_accs_iris_gamma = explore_gamma_parameter(
    X_iris_train, X_iris_test, y_iris_train, y_iris_test, "鸢尾花数据集", best_c_iris
)

In [ ]:
# 📊 任务1.3: 参数交互效应分析
# 🎯 目标：发现C和gamma参数之间的交互作用

def analyze_parameter_interaction(X_train, X_test, y_train, y_test, dataset_name):
    """分析C和gamma参数的交互效应"""
    
    # TODO: 定义参数网格
    c_values = [0.1, 1, 10, 100]
    gamma_values = [0.01, 0.1, 1, 10]
    
    # 创建结果矩阵
    results_matrix = np.zeros((len(c_values), len(gamma_values)))
    
    print(f"🔍 正在分析{dataset_name}上参数的交互效应...")
    
    # 网格搜索
    for i, c in enumerate(c_values):
        for j, gamma in enumerate(gamma_values):
            svm = SVC(C=c, kernel='rbf', gamma=gamma, random_state=42)
            svm.fit(X_train, y_train)
            acc = accuracy_score(y_test, svm.predict(X_test))
            results_matrix[i, j] = acc
            print(f"C={c}, gamma={gamma}: 准确率={acc:.4f}")
    
    # 绘制热力图
    plt.figure(figsize=(10, 8))
    sns.heatmap(results_matrix, annot=True, fmt='.3f', cmap='YlOrRd',
                xticklabels=gamma_values, yticklabels=c_values)
    plt.xlabel('Gamma值')
    plt.ylabel('C值')
    plt.title(f'{dataset_name}: C和Gamma参数交互效应热力图')
    plt.show()
    
    # 找到最佳参数组合
    best_i, best_j = np.unravel_index(np.argmax(results_matrix), results_matrix.shape)
    best_c = c_values[best_i]
    best_gamma = gamma_values[best_j]
    best_acc = results_matrix[best_i, best_j]
    
    print(f"\n🏆 最佳参数组合: C={best_c}, gamma={best_gamma}")
    print(f"📈 最佳准确率: {best_acc:.4f}")
    
    # TODO: 分析交互效应
    print("\n💡 交互效应分析：")
    print("1. 是否存在某些参数组合特别差？（参数陷阱）")
    print("2. C和gamma是否存在相互补偿效应？")
    print("3. 不同C值下，最优gamma值如何变化？")
    
    return results_matrix, (best_c, best_gamma, best_acc)

print("\n" + "="*60)
print("🎯 探索任务1.3: 参数交互效应分析")
print("="*60)

# 分析乳腺癌数据集
matrix_bc, best_params_bc = analyze_parameter_interaction(
    X_bc_train, X_bc_test, y_bc_train, y_bc_test, "乳腺癌数据集"
)

# 分析鸢尾花数据集
matrix_iris, best_params_iris = analyze_parameter_interaction(
    X_iris_train, X_iris_test, y_iris_train, y_iris_test, "鸢尾花数据集"
)

## 🎯 探索任务2：算法的适用边界

### 🤔 思考问题：
1. **算法在什么条件下会失效？**
2. **数据质量如何影响算法性能？**
3. **不同算法对数据特性的敏感度如何？**

### 🔬 实验任务：
通过模拟真实世界的问题场景，测试算法的鲁棒性

In [ ]:
# 📊 任务2.1: 数据量对性能的影响
# 🎯 目标：了解算法在数据稀少时的表现

def test_data_sufficiency(X_train, X_test, y_train, y_test, dataset_name):
    """测试不同数据量下的算法性能"""
    
    # TODO: 定义不同的训练数据比例
    data_ratios = [1.0, 0.8, 0.6, 0.4, 0.2, 0.1]  # 100%到10%
    
    algorithms = {
        '逻辑回归': LogisticRegression(random_state=42, max_iter=1000),
        '线性SVM': SVC(C=1, kernel='linear', random_state=42),
        'RBF_SVM': SVC(C=1, kernel='rbf', gamma='scale', random_state=42)
    }
    
    results = {alg: [] for alg in algorithms.keys()}
    
    print(f"🔍 正在测试{dataset_name}上数据量的影响...")
    
    for ratio in data_ratios:
        # 随机采样指定比例的数据
        n_samples = int(len(X_train) * ratio)
        indices = np.random.choice(len(X_train), n_samples, replace=False)
        X_subset = X_train[indices]
        y_subset = y_train[indices]
        
        print(f"\n📊 使用{ratio*100:.0f}%的数据 ({n_samples}个样本):")
        
        for alg_name, alg in algorithms.items():
            alg.fit(X_subset, y_subset)
            acc = accuracy_score(y_test, alg.predict(X_test))
            results[alg_name].append(acc)
            print(f"  {alg_name}: {acc:.4f}")
    
    # 绘制结果
    plt.figure(figsize=(12, 8))
    for alg_name, accs in results.items():
        plt.plot([r*100 for r in data_ratios], accs, 'o-', label=alg_name, linewidth=2, markersize=8)
    
    plt.xlabel('训练数据比例 (%)')
    plt.ylabel('测试准确率')
    plt.title(f'{dataset_name}: 数据量对算法性能的影响')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # TODO: 分析结果
    print("\n💡 数据量影响分析：")
    print("1. 哪个算法在小数据集上表现最好？")
    print("2. 哪个算法对数据量最敏感？")
    print("3. 在多少数据量时性能趋于稳定？")
    print("4. 这对实际应用有什么启示？")
    
    return results

print("\n" + "="*60)
print("🎯 探索任务2.1: 数据量对性能的影响")
print("="*60)

# 在乳腺癌数据集上测试
data_sufficiency_bc = test_data_sufficiency(
    X_bc_train, X_bc_test, y_bc_train, y_bc_test, "乳腺癌数据集"
)

# 在鸢尾花数据集上测试
data_sufficiency_iris = test_data_sufficiency(
    X_iris_train, X_iris_test, y_iris_train, y_iris_test, "鸢尾花数据集"
)

In [ ]:
# 📊 任务2.2: 噪声敏感性测试
# 🎯 目标：了解算法对噪声数据的鲁棒性

def add_noise(X, noise_level=0.1, noise_type='gaussian'):
    """向数据添加噪声"""
    if noise_type == 'gaussian':
        noise = np.random.normal(0, noise_level, X.shape)
    elif noise_type == 'uniform':
        noise = np.random.uniform(-noise_level, noise_level, X.shape)
    return X + noise

def test_noise_robustness(X_train, X_test, y_train, y_test, dataset_name):
    """测试算法对噪声的鲁棒性"""
    
    # TODO: 定义不同的噪声水平
    noise_levels = [0.0, 0.05, 0.1, 0.2, 0.3, 0.5]
    
    algorithms = {
        '逻辑回归': LogisticRegression(random_state=42, max_iter=1000),
        '线性SVM': SVC(C=1, kernel='linear', random_state=42),
        'RBF_SVM': SVC(C=1, kernel='rbf', gamma='scale', random_state=42)
    }
    
    results = {alg: [] for alg in algorithms.keys()}
    
    print(f"🔍 正在测试{dataset_name}上噪声鲁棒性...")
    
    for noise_level in noise_levels:
        # 向训练数据添加噪声
        X_train_noisy = add_noise(X_train, noise_level)
        
        print(f"\n📊 噪声水平 {noise_level:.1f}:")
        
        for alg_name, alg in algorithms.items():
            alg.fit(X_train_noisy, y_train)
            acc = accuracy_score(y_test, alg.predict(X_test))
            results[alg_name].append(acc)
            print(f"  {alg_name}: {acc:.4f}")
    
    # 绘制结果
    plt.figure(figsize=(12, 8))
    for alg_name, accs in results.items():
        plt.plot(noise_levels, accs, 'o-', label=alg_name, linewidth=2, markersize=8)
    
    plt.xlabel('噪声水平')
    plt.ylabel('测试准确率')
    plt.title(f'{dataset_name}: 噪声对算法性能的影响')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 计算性能下降率
    print("\n💡 噪声敏感性分析：")
    for alg_name, accs in results.items():
        original_acc = accs[0]
        final_acc = accs[-1]
        drop_rate = (original_acc - final_acc) / original_acc * 100
        print(f"{alg_name}: 性能下降 {drop_rate:.1f}%")
    
    print("\n思考问题：")
    print("1. 哪个算法对噪声最鲁棒？")
    print("2. 为什么某些算法对噪声更敏感？")
    print("3. 在实际应用中如何应对噪声数据？")
    
    return results

print("\n" + "="*60)
print("🎯 探索任务2.2: 噪声敏感性测试")
print("="*60)

# 在乳腺癌数据集上测试
noise_robustness_bc = test_noise_robustness(
    X_bc_train, X_bc_test, y_bc_train, y_bc_test, "乳腺癌数据集"
)

# 在鸢尾花数据集上测试
noise_robustness_iris = test_noise_robustness(
    X_iris_train, X_iris_test, y_iris_train, y_iris_test, "鸢尾花数据集"
)

## 🎯 探索任务3：真实问题挑战

### 🤔 思考问题：
1. **当类别不平衡时会发生什么？**
2. **高维数据会带来什么挑战？**
3. **如何评估算法在真实场景中的表现？**

### 🔬 实验任务：
模拟真实世界中的复杂情况，测试算法的适应性

In [ ]:
# 📊 任务3.1: 类别不平衡问题
# 🎯 目标：了解算法在不平衡数据上的表现和解决方案

def create_imbalanced_dataset(X, y, minority_ratio=0.1):
    """创建不平衡数据集"""
    # 假设我们让类别0为多数类，类别1为少数类
    majority_mask = y == 0
    minority_mask = y == 1
    
    # 计算需要的少数类样本数
    n_minority = int(len(y[majority_mask]) * minority_ratio / (1 - minority_ratio))
    
    # 随机选择少数类样本
    minority_indices = np.where(minority_mask)[0]
    selected_minority = np.random.choice(minority_indices, 
                                        min(n_minority, len(minority_indices)), 
                                        replace=False)
    
    # 组合数据
    majority_indices = np.where(majority_mask)[0]
    selected_indices = np.concatenate([majority_indices, selected_minority])
    
    X_imbalanced = X[selected_indices]
    y_imbalanced = y[selected_indices]
    
    return X_imbalanced, y_imbalanced

def test_imbalanced_learning(X_train, X_test, y_train, y_test, dataset_name):
    """测试类别不平衡下的学习性能"""
    
    # 创建不同不平衡程度的数据集
    imbalance_ratios = [0.5, 0.3, 0.2, 0.1, 0.05]  # 少数类比例
    
    algorithms = {
        '逻辑回归': LogisticRegression(random_state=42, max_iter=1000),
        '线性SVM': SVC(C=1, kernel='linear', random_state=42),
        'RBF_SVM': SVC(C=1, kernel='rbf', gamma='scale', random_state=42)
    }
    
    # 只在乳腺癌数据集上进行（因为它是二分类）
    if len(np.unique(y_train)) != 2:
        print(f"跳过 {dataset_name}（非二分类数据集）")
        return None
    
    results = {ratio: {} for ratio in imbalance_ratios}
    
    print(f"🔍 正在测试{dataset_name}上类别不平衡问题...")
    
    original_acc = {}
    
    for ratio in imbalance_ratios:
        # 创建不平衡数据集
        X_train_imb, y_train_imb = create_imbalanced_dataset(X_train, y_train, ratio)
        
        print(f"\n📊 少数类比例 {ratio:.2f} (总样本: {len(X_train_imb)})")
        unique, counts = np.unique(y_train_imb, return_counts=True)
        for cls, cnt in zip(unique, counts):
            print(f"  类别 {cls}: {cnt} 样本")
        
        for alg_name, alg in algorithms.items():
            alg.fit(X_train_imb, y_train_imb)
            y_pred = alg.predict(X_test)
            
            # 计算多种评估指标
            from sklearn.metrics import precision_score, recall_score, f1_score
            
            acc = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            f1 = f1_score(y_test, y_pred, average='weighted')
            
            results[ratio][alg_name] = {
                'accuracy': acc,
                'precision': precision,
                'recall': recall,
                'f1': f1
            }
            
            print(f"  {alg_name}: ACC={acc:.3f}, P={precision:.3f}, R={recall:.3f}, F1={f1:.3f}")
    
    # 绘制结果
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    metric_names = ['准确率', '精确率', '召回率', 'F1分数']
    
    for idx, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
        ax = axes[idx//2, idx%2]
        for alg_name in algorithms.keys():
            values = [results[ratio][alg_name][metric] for ratio in imbalance_ratios]
            ax.plot(imbalance_ratios, values, 'o-', label=alg_name, linewidth=2, markersize=8)
        
        ax.set_xlabel('少数类比例')
        ax.set_ylabel(metric_name)
        ax.set_title(f'{dataset_name}: 类别不平衡对{metric_name}的影响')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 类别不平衡分析：")
    print("1. 哪个指标受类别不平衡影响最大？")
    print("2. 哪个算法对类别不平衡最鲁棒？")
    print("3. 为什么准确率可能会产生误导？")
    print("4. 在实际应用中如何处理类别不平衡问题？")
    
    return results

print("\n" + "="*60)
print("🎯 探索任务3.1: 类别不平衡问题")
print("="*60)

# 只在乳腺癌数据集上测试（二分类）
imbalance_results = test_imbalanced_learning(
    X_bc_train, X_bc_test, y_bc_train, y_bc_test, "乳腺癌数据集"
)

## 📝 实验总结与反思

### 🎯 您的发现记录：

通过以上探索实验，请记录您的关键发现：

**1. 参数理解方面：**
- C参数的本质作用是什么？
- Gamma参数如何影响模型复杂度？
- 参数之间存在什么样的交互效应？

**2. 算法特性方面：**
- 哪些算法适合小样本学习？
- 哪些算法对噪声更鲁棒？
- 算法的适用边界是什么？

**3. 实际应用方面：**
- 如何选择合适的算法？
- 如何评估算法在真实场景中的表现？
- 面对数据质量问题该如何应对？

### 🚀 下一步探索方向：
1. **特征工程实验** - 尝试不同的特征处理方法
2. **集成学习实验** - 结合多个算法的预测结果
3. **自定义损失函数** - 针对特定问题优化目标函数
4. **实际数据集应用** - 在真实数据上验证您的发现

### 💡 深度学习建议：
- **保持好奇心** - 总是问"为什么"和"怎么样"
- **记录实验过程** - 建立自己的知识库
- **挑战现有结论** - 提出假设并验证
- **联系实际应用** - 思考理论在实践中的意义

**记住**：真正的学习不是完成任务，而是通过实验获得对算法本质的深刻理解！